# Playground

In [1]:
from paoc.helper import get_input, parse_multiline_string as pms
lines = get_input()

In [2]:
ex_lines = pms("""
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
""")

In [3]:
import re
import itertools
from tqdm import tqdm

In [4]:
def binary_combinations(indices: list[int]) -> list[dict[int, tuple[str, str]]]:
    return [{i: v for i, v in zip(indices, comb)} for comb in itertools.product(['.', '#'], repeat=len(indices))]

doc = ex_lines
res = 0
for line in doc:
    springs, groups = line.split()
    groups = tuple(int(group) for group in groups.split(','))
    indices = [i for i, s in enumerate(springs) if s == '?']
    valid_combs = 0
    for comb in binary_combinations(indices):
        springs_ = list(springs)
        for i, char in comb.items():
            springs_[i] = char
        groups_ = tuple(group.end()-group.start() for group in re.finditer('#+', ''.join(springs_)))
        valid_combs += groups_ == groups
    res += valid_combs
res

21

In [5]:
from functools import cache

O, D = '.', '#'

def get_blocks(springs: str) -> tuple[int]:
    blocks = []
    size = 0
    for spring in springs:
        if spring == D:
            size += 1
        else:
            if size:
                blocks.append(size)
            size = 0
    if size:
        blocks.append(size)
    blocks = tuple(blocks)
    return blocks

@cache
def count_arrangements_1(blocks: tuple[int], springs: str, current: str) -> int:
    """
    - blocks is the signature of '#'-block lengths we want to match against
    - springs is the original line
    - current is the version of springs we're building
    """
    i = len(current)
    if i == len(springs):
        c_blocks = get_blocks(current)
        return c_blocks == blocks
    else:
        # early stop
        if current and current[i-1] == '.':
            c_blocks = get_blocks(current)
            if c_blocks and c_blocks != blocks[:len(c_blocks)]:
                return 0
    
    count = 0
    char = springs[i]

    if char == '?':
        count += count_arrangements_1(blocks, springs, current + '.')
        count += count_arrangements_1(blocks, springs, current + '#')
    else:
        count += count_arrangements_1(blocks, springs, current + char)

    return count

In [6]:
doc = lines
res = 0
for line in tqdm(doc, total=len(doc)):
    springs, blocks = line.split()
    blocks = tuple(int(block) for block in blocks.split(','))
    n_valid = count_arrangements_1(blocks, springs, '')
    res += n_valid
res

100%|██████████| 1000/1000 [00:00<00:00, 3122.27it/s]


7490

In [7]:
doc = ex_lines
res = 0
for line in tqdm(doc, total=len(doc)):
    springs, groups = line.split()
    groups = tuple(int(group) for group in groups.split(','))
    # for part 2, we multiply both springs and groups by 5
    springs = '?'.join([springs]*5)
    groups = groups * 5
    n_valid = count_arrangements_1(groups, springs, '')
    res += n_valid
res

100%|██████████| 6/6 [00:19<00:00,  3.18s/it]


525152

In [8]:
from functools import cache

@cache
def count_arrangements_2(springs: str, blocks: tuple[int]) -> int:
    """ first call is with original springs string, and all blocks, they both get shorter until consumed """
    # no more springs left
    if springs == '':
        return blocks == ()
    # no more blocks left
    if blocks == ():
        return '#' not in springs
    
    count = 0

    if springs[0] in '.?':
        # nothing happens to the blocks, we just move on to the next index of springs
        count += count_arrangements_2(springs[1:], blocks)
    if springs[0] in '#?':
        # check if enough springs left for current block
        if blocks[0] <= len(springs):
            # check if all next n springs are actually broken (or might be broken)
            if '.' not in springs[:blocks[0]]:
                # check if spring after that is operational (or might be) (or there is no spring after)
                if blocks[0] == len(springs) or springs[blocks[0]] in '.?':
                    count += count_arrangements_2(springs[blocks[0] + 1:], blocks[1:])
        
    return count


doc = lines
res = 0
for line in tqdm(doc, total=len(doc)):
    springs, groups = line.split()
    groups = tuple(map(int, groups.split(',')))
    # for part 2, we multiply both springs and groups by 5
    springs = '?'.join([springs]*5)
    groups = groups * 5
    n_valid = count_arrangements_2(springs, groups)
    res += n_valid
res

100%|██████████| 1000/1000 [00:00<00:00, 1119.82it/s]


65607131946466